# initialization

import the packages and start the GUI

In [1]:
from TableTennisEnvironmentV0.TableTennisEnvironment import TableTennisEnv
from utils.BotPluginMoveIt import BotPluginMoveIt
import pybullet as p
import numpy as np
import time
import joblib
from utils.physics_solvers import estimateInitVelocity, estimate_hitting_point, calculate_delta, had_double_bounce, had_no_bounce
import math
import torch
import torch.nn as nn
from delta_estimator_mlp.DeltaEstimatorMLP import DeltaEstimatorMLP

env = TableTennisEnv(show_gui=False)

pybullet build time: Nov 28 2023 23:51:11
/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Predicted Output: [-0.2558609 -1.5050874]


# MoveIt Plugin Test

moveit plugin allows pybullet to communicate with MOVEIT. Currently it provides 2 functions: plan_motion, execute_plan

In [2]:
moveit_plugin = BotPluginMoveIt(environment = env)

[ERROR] [1718203603.603526613]: Group 'paddle_group' for end-effector 'paddle' cannot be its own parent
[ WARN] [1718203603.636865361]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
[ WARN] [1718203603.648154591]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
[ WARN] [1718203603.666501891]: The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.


In [3]:
moveit_plugin.plan_motion(x = -0.4,
                          y = -0.3,
                          z = 0.6,
                          roll = 0,
                          pitch = 0,
                          yaw = np.pi/2)
moveit_plugin.execute_plan()

In [4]:
end_effector_link_index = 6
base_link_index = 0
state_end_effector = p.getLinkState(env._robotic_arm, end_effector_link_index)
state_base = p.getLinkState(env._robotic_arm, base_link_index)

# Position and orientation of the end effector
position_end_effector = state_end_effector[0]
position_base = state_base[0]

print([position_end_effector[i] - position_base[i] for i in range(len(position_end_effector))])

[-0.4541817912027062, -0.27390822115130015, 0.4985942128480132]


# Load Delta Estimator V1

Delta estimator V1 is built on random forest regression. 

In [5]:
model_filename = 'delta_regressor_RF.joblib'
regressor2 = joblib.load(model_filename)
print("Model loaded successfully")

Model loaded successfully


/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.1.post1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.1.post1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Load Delta Estimator V2

Delta Estimator V2 is built on neural nets

In [6]:
# Load the scaler
scaler = joblib.load('delta_estimator_mlp/scaler.pkl')

# Parameters
input_size = 9  # Number of input features
hidden_size = 32  # Number of neurons in the hidden layer
output_size = 2  # Number of output features (delta_y, delta_z)

# Create model and load trained weights
model = DeltaEstimatorMLP(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('delta_estimator_mlp/delta_estimator_mlp.pth'))
model.eval()

# Sample inference code
def calculate_delta_v2(input_features):
    # Convert input features to tensor
    input_tensor = torch.tensor(input_features, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        prediction = model(input_tensor)
    
    # Inverse transform the prediction to get original values
    prediction_original = scaler.inverse_transform(prediction.numpy())
    
    return prediction_original[0]

# Example input features for inference
sample_input = np.array([
    0.5, 0.5, 0.5,  # ball_initial_pos_x, ball_initial_pos_y, ball_initial_pos_z
    1.0,  # distance_ball_robot
    0.1, 0.1, 0.1,  # estimated_initial_velocity_x, estimated_initial_velocity_y, estimated_initial_velocity_z
    0.2, 0.2  # estimated_hitting_point_y, estimated_hitting_point_z
])

# Perform inference and print the result
predicted_output = calculate_delta_v2(sample_input)
print("Predicted Output:", predicted_output)


Predicted Output: [-0.2558609 -1.5050874]


/home/n3r0/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Paddle Goal Position Calculator

This function calculates the estimated hitting point for the ball and the paddle.

In [7]:
def find_goal_position():
    ball_position1 = ball_initial_position = p.getBasePositionAndOrientation(env._ball)[0]
    robot_initial_position = p.getLinkState(env._robotic_arm, 0)[0]      # can be collected from optitrack
    distance = robot_initial_position[0] - ball_initial_position[0]
    action = [0, 0, 0, 0, 0, 0]
    env.step(action)
    ball_position2 = p.getBasePositionAndOrientation(env._ball)[0]
    
    table_height = 1.0
    estimated_initial_velocity = estimateInitVelocity(ball_position1, ball_position2, 1/240)
    estimated_hitting_point, t1, t2 = estimate_hitting_point(ball_initial_position, robot_initial_position, estimated_initial_velocity, table_height)
    # input_for_delta = [[ball_initial_position[0], ball_initial_position[1], ball_initial_position[2], distance, estimated_initial_velocity[0], estimated_initial_velocity[1], estimated_initial_velocity[2], estimated_hitting_point[0], estimated_hitting_point[1]]]
    input_for_delta = [ball_initial_position[0], ball_initial_position[1], ball_initial_position[2], distance, estimated_initial_velocity[0], estimated_initial_velocity[1], estimated_initial_velocity[2], estimated_hitting_point[0], estimated_hitting_point[1]]
    # print(input_for_delta)
    # delta = regressor2.predict(np.asarray(input_for_delta)) # this uses random forest model
    delta = calculate_delta_v2(np.asarray(input_for_delta))
    return estimated_hitting_point+delta, estimated_initial_velocity

Ball position range:
- x : [-2, -1]
- y : [-0.5, +0.5]
- z : [1.5, 2.5]

Ball velocity range:
- x : [4, 8]
- y : [-0.5, +0.5]
- z : [-3, +1]
    

This function calculates the distance between the ball and the paddle

In [8]:

def ball_paddle_distance(ball_id, robot_id): ## defining probability = 1/ distance between ball and paddle
    paddle_position = p.getLinkState(robot_id, 6)[0]
    ball_position, _ = p.getBasePositionAndOrientation(ball_id)
    distance = math.sqrt((paddle_position[0] - ball_position[0])**2 +
                        (paddle_position[1] - ball_position[1])**2 +
                        (paddle_position[2] - ball_position[2])**2)
    return distance

In [9]:
ball_paddle_distance(env._ball, env._robotic_arm)

3.329259502538517

# Collision Checker

This function checks collision between two bodies. It needs to be checked immediately after stepSimulation. 

In [10]:
def check_collision(body1, body2):
    contacts = p.getContactPoints(body1, body2)
    return len(contacts)>0

# The Loop

In [11]:
## define states
BALL_IS_THROWN = 0
BALL_HIT_THE_PADDLE = 1
BALL_HIT_THE_TABLE = 2

In [12]:
def the_loop(ball_init_position = [-1.3, -0.2, 2.1], 
             real_initial_velocity = [7.0, -0.0, -2], 
             paddle_hitting_vel = [-1000*np.pi/180, -900*np.pi/180, 900*np.pi/180]): # for joint 3, 4, 5
    ## Throw the ball
    env.get_new_ball(position=ball_init_position)

    distance_ball_robot = p.getLinkState(env._robotic_arm, 0)[0][0] - ball_init_position[0]
    
    # print('I will throw ball')
    env.throw_ball(real_initial_velocity)
    state = BALL_IS_THROWN
    table_hit_point = None
    # print('I have thrown ball')

    # print('starting to find goal position')
    goal_position, estimated_init_vel = find_goal_position()
    # print(goal_position)
    # print('have calculated goal position')
    # print(goal_position[0])
    # print('starting to go to the goal position')
    if goal_position[1] > 1.5:
        moveit_plugin.plan_motion(x = -0.5,
                              y = goal_position[0],
                              z = goal_position[1] - 1.4,
                              roll = 0,
                              pitch = 0,
                              yaw = np.pi/2)
        moveit_plugin.execute_plan()
        # for step in moveit_plugin.plan[1].joint_trajectory.points:
        #     joint_angles = step.positions
        #     for i, angle in enumerate(joint_angles):
        #         p.setJointMotorControl2(bodyIndex=env._robotic_arm,
        #                                 jointIndex=i,
        #                                 controlMode=p.POSITION_CONTROL,
        #                                 targetPosition=angle,
        #                                )
        #         p.stepSimulation()
        #         time.sleep(1/240.0)
    
        for i in range(500):
            
            # print(ball_paddle_distance(env._ball, env._robotic_arm))
            if state == BALL_IS_THROWN and check_collision(env._ball, env._robotic_arm):
                state = BALL_HIT_THE_PADDLE
                print('ball has hit the paddle')
    
                # p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                #                         jointIndex=4,
                #                         controlMode=p.POSITION_CONTROL,
                #                         targetPosition=80*np.pi/180,
                #                        )
                # p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                #                         jointIndex=5,
                #                         controlMode=p.POSITION_CONTROL,
                #                         targetPosition=220*np.pi/180,
                #                        )
                
    
                # p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                #                         jointIndex=3,
                #                         controlMode=p.POSITION_CONTROL,
                #                         targetPosition=0*np.pi/180,
                #                        )
    
                ## generate impact force
    
                for timer in range(15):
    
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=3,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[0],
                                           )
    
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=4,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[1],
                                           )
                    p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=5,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=paddle_hitting_vel[2],
                                           )
                    time.sleep(1/240.0)
                    p.stepSimulation()
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=3,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=0*np.pi/180,
                                           )
                
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                            jointIndex=4,
                                            controlMode=p.VELOCITY_CONTROL,
                                            targetVelocity=0*np.pi/180,
                                           )
                p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                                        jointIndex=5,
                                        controlMode=p.VELOCITY_CONTROL,
                                        targetVelocity=0*np.pi/180,
                                       )
                # p.setJointMotorControl2(bodyIndex=env._robotic_arm,
                #                         jointIndex=3,
                #                         controlMode=p.POSITION_CONTROL,
                #                         targetPosition=0*np.pi/180,
                #                        )
    
            if state == BALL_HIT_THE_PADDLE and check_collision(env._ball, env._table_body_id):
                state = BALL_HIT_THE_TABLE
                table_hit_point = env.get_state()['ball']['position']
                print('ball has hit the table at: ', table_hit_point)
            time.sleep(1/240.0)
            p.stepSimulation()
                
        # moveit_plugin.group.go(wait = True)
        # moveit_plugin.execute_plan()
    return estimated_init_vel, goal_position, distance_ball_robot, table_hit_point
    
the_loop()

([6.990339872011777, 0.0, -2.038114963431923],
 array([-0.19381847,  1.54570123]),
 3.0046014920448467,
 None)

In [13]:
p.getJointState(env._robotic_arm, 5)

(3.285519128681576,
 4.349298698969051e-14,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 -0.11893137830764115)

# prepare a bigger table to collect data

In [14]:
p.removeBody(env._table_body_id) # removing the old table

In [15]:
# load the new big table
table_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[5, 5, 0.5])
table_visual_id = -1  # No visual representation for now
env._table_body_id = p.createMultiBody(baseMass=20000.0, baseCollisionShapeIndex=table_id, baseVisualShapeIndex=table_visual_id, basePosition=[0, 0, 0.7])
p.changeDynamics(env._table_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)
# p.changeDynamics(self._table_body_id, -1, restitution=1.0)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)

In [16]:
the_loop(ball_init_position = [-1.3, -0.4, 2.1], 
         real_initial_velocity = [7.0, -0.0, -2])

([6.990339872011777, 0.0, -2.038114963431923],
 array([-0.39980575,  1.61260182]),
 3.0034298677148925,
 None)

params to be considered:
- ball_init_pos_x
- ball_init_pos_y
- ball_init_pos_z
- ball_init_vel_estimated_x
- ball_init_vel_estimated_y
- ball_init_vel_estimated_z
- physics_delta_hitting_point_y
- physics_delta_hitting_point_z
- distance_ball_robot
- ball_hitting_success
- robot_angular_vel_joint3
- robot_angular_vel_joint4
- robot_angular_vel_joint5
- table_hitting_pos_x
- table_hitting_pos_y

# Collect Data

In [17]:
import csv
import random
import numpy as np
import pybullet as p

# Define the ranges for the inputs
ball_init_position_x_range = [-2, -1]
ball_init_position_y_range = [-0.5, 0.5]
ball_init_position_z_range = [1.5, 2.5]

real_initial_velocity_x_range = [4, 8]
real_initial_velocity_y_range = [-0.5, 0.5]
real_initial_velocity_z_range = [-3, 1]

paddle_hitting_vel_joint3_range = [-1200, 1200]
paddle_hitting_vel_joint4_range = [-1200, 1200]
paddle_hitting_vel_joint5_range = [-1200, 1200]

def generate_random_inputs():
    ball_init_position = [
        random.uniform(*ball_init_position_x_range),
        random.uniform(*ball_init_position_y_range),
        random.uniform(*ball_init_position_z_range)
    ]
    real_initial_velocity = [
        random.uniform(*real_initial_velocity_x_range),
        random.uniform(*real_initial_velocity_y_range),
        random.uniform(*real_initial_velocity_z_range)
    ]
    paddle_hitting_vel = [
        random.uniform(*paddle_hitting_vel_joint3_range) * np.pi / 180,
        random.uniform(*paddle_hitting_vel_joint4_range) * np.pi / 180,
        random.uniform(*paddle_hitting_vel_joint5_range) * np.pi / 180
    ]
    return ball_init_position, real_initial_velocity, paddle_hitting_vel

def run_simulation_and_store_results(filename, num_iterations):
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        # writer.writerow([
        #     "ball_init_position_x", "ball_init_position_y", "ball_init_position_z",
        #     "estimated_init_vel_x", "estimated_init_vel_y", "estimated_init_vel_z",
        #     "goal_position_y", "goal_position_z",
        #     "distance_ball_robot",
        #     "table_hit_point_x", "table_hit_point_y", "table_hit_point_z",
        #     "paddle_hitting_vel_joint3", "paddle_hitting_vel_joint4", "paddle_hitting_vel_joint5"
        # ])

        for _ in range(num_iterations):

            ### get fresh table and ball
            p.removeBody(env._table_body_id) # removing the old table
            # load the new big table            table_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[5, 5, 0.5])
            table_visual_id = -1  # No visual representation for now
            env._table_body_id = p.createMultiBody(baseMass=20000.0, baseCollisionShapeIndex=table_id, baseVisualShapeIndex=table_visual_id, basePosition=[0, 0, 0.7])
            p.changeDynamics(env._table_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)
            # p.changeDynamics(self._table_body_id, -1, restitution=1.0)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)
            
            p.removeBody(env._ball)
            
            ball_init_position, real_initial_velocity, paddle_hitting_vel = generate_random_inputs()
            estimated_init_vel, goal_position, distance_ball_robot, table_hit_point = the_loop(
                ball_init_position=ball_init_position,
                real_initial_velocity=real_initial_velocity,
                paddle_hitting_vel=paddle_hitting_vel
            )

            # Ensure that table_hit_point is not None
            table_hit_point_x, table_hit_point_y, table_hit_point_z = table_hit_point if table_hit_point else (None, None, None)
            if not table_hit_point_x == None:
                # Write the results
                writer.writerow([
                    ball_init_position[0], 
                    ball_init_position[1], 
                    ball_init_position[2],
                    estimated_init_vel[0], 
                    estimated_init_vel[1], 
                    estimated_init_vel[2],
                    goal_position[0], 
                    goal_position[1],
                    distance_ball_robot,
                    table_hit_point_x, 
                    table_hit_point_y, 
                    table_hit_point_z,
                    paddle_hitting_vel[0], 
                    paddle_hitting_vel[1], 
                    paddle_hitting_vel[2]
                ])

# Example usage
run_simulation_and_store_results('hitting_velocity_simulation_data.csv', 10000)

ball has hit the paddle
ball has hit the table at:  (-1.7573184496518581, -0.562700073909783, 1.2203425624325166)
ball has hit the paddle
ball has hit the table at:  (1.0917658011267612, -0.3496564021963942, 1.2205055329525165)
ball has hit the paddle
ball has hit the table at:  (1.0425832078882613, -0.27050240104804907, 1.2189647255651557)
ball has hit the paddle
ball has hit the table at:  (1.023470667548762, -0.49197459987898257, 1.2223310799142182)
ball has hit the paddle
ball has hit the table at:  (1.8691043266951484, 0.3041352103198928, 1.2221596450717167)
ball has hit the paddle
ball has hit the table at:  (2.2797209575842454, -0.2989368580112906, 1.2299883300791297)
ball has hit the paddle
ball has hit the table at:  (1.120008857721466, 0.7650999937657181, 1.222936046291417)
ball has hit the paddle
ball has hit the table at:  (1.1325872217792654, 0.19581110839756502, 1.2276792613021927)
ball has hit the paddle
ball has hit the table at:  (1.5655451536760445, 0.6948322598674472

KeyboardInterrupt: 